<a href="https://colab.research.google.com/github/tarakantaacharya/NLPinternal/blob/main/rag_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
! pip install langchain
! pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.3 MB/s eta 0:0

In [ ]:
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 17.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df_ap = pd.read_csv('/content/ap_eapcet-last-rank-2023.csv')
df_ts = pd.read_csv('/content/TSEAMCET_2023_FINALPHASE.csv')

In [ ]:
df_ap['State'] = "Andhra Pradesh"
df_ts['State'] = "Telangana"

In [ ]:
df_ts.columns

Index(['Inst\n Code', 'Institute Name', 'Place', 'Dist \nCode', 'Co Education',
       'College Type', 'Year of Estab', 'Branch Code', 'Branch Name',
       'OC \nBOYS', 'OC \nGIRLS', 'BC_A \nBOYS', 'BC_A \nGIRLS', 'BC_B \nBOYS',
       'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS', 'BC_D \nBOYS',
       'BC_D \nGIRLS', 'BC_E \nBOYS', 'BC_E \nGIRLS', 'SC \nBOYS',
       'SC \nGIRLS', 'ST \nBOYS', 'ST \nGIRLS', 'EWS \nGEN OU',
       'EWS \nGIRLS OU', 'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.columns

Index(['SNO', 'INSTCODE', 'NAME OF THE INSTITUTION', 'TYPE', 'INST\r_RE\rG',
       'DIST', 'PLACE', 'COED', 'AFFL.', 'ESTD', 'A_RE\rG', 'branch_\rcode',
       'OC_BO\rYS', 'OC_GIR\rLS', 'SC_BO\rYS', 'SC_GIR\rLS', 'ST_BO\rYS',
       'ST_GIR\rLS', 'BCA_B\rOYS', 'BCA_GI\rRLS', 'BCB_B\rOYS', 'BCB_GI\rRLS',
       'BCC_B\rOYS', 'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BCE_B\rOYS',
       'BCE_GI\rRLS', 'OC_EWS_B\rOYS', 'OC_EWS_G\rIRLS', 'Unnamed: 30',
       'State'],
      dtype='object')

In [ ]:
df_ts.rename(columns={
    'Inst\n Code': 'Institute Code',
    'Institute Name': 'Institute Name',
    'College Type': 'College Type',
    'Place': 'Location',
    'Dist \nCode': 'District Code',
    'Co Education': 'Co-Ed Status',
    'Year of Estab': 'Established Year',
    'Branch Code': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Tuition Fee': 'Tuition Fee',
    'Affiliated To': 'Affiliated To',
    'EWS \nGEN OU': 'EWS Boys',
    'EWS \nGIRLS OU': 'EWS Girls',
    'OC \nBOYS': 'OC Boys',
    'OC \nGIRLS': 'OC Girls',
    'SC \nBOYS': 'SC Boys',
    'SC \nGIRLS': 'SC Girls',
    'ST \nBOYS': 'ST Boys',
    'ST \nGIRLS': 'ST Girls',
    'BC_A \nBOYS': 'BC-A Boys',
    'BC_A \nGIRLS': 'BC-A Girls',
    'BC_E \nBOYS': 'BC-E Boys',
    'BC_E \nGIRLS': 'BC-E Girls'
}, inplace=True)

In [ ]:
df_ts.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS',
       'BC_D \nBOYS', 'BC_D \nGIRLS', 'BC-E Boys', 'BC-E Girls', 'SC Boys',
       'SC Girls', 'ST Boys', 'ST Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'Affiliated To', 'State'],
      dtype='object')

In [ ]:
df_ap.rename(columns={
    'INSTCODE': 'Institute Code',
    'NAME OF THE INSTITUTION': 'Institute Name',
    'TYPE': 'College Type',
    'PLACE': 'Location',
    'DIST': 'District Code',
    'COED': 'Co-Ed Status',
    'ESTD': 'Established Year',
    'branch_\rcode': 'Branch Code',
    'Branch Name': 'Branch Name',
    'Unnamed: 30': 'Tuition Fee',
    'AFFL.': 'Affiliated To',
    'OC_EWS_B\rOYS': 'EWS Boys',
    'OC_EWS_G\rIRLS': 'EWS Girls',
    'BCA_B\rOYS': 'BC-A Boys',
    'BCA_GI\rRLS': 'BC-A Girls',
    'BCE_B\rOYS': 'BC-E Boys',
    'BCE_GI\rRLS': 'BC-E Girls',
    'OC_BO\rYS': 'OC Boys',
    'OC_GIR\rLS': 'OC Girls',
    'SC_BO\rYS': 'SC Boys',
    'SC_GIR\rLS': 'SC Girls',
    'ST_BO\rYS': 'ST Boys',
    'ST_GIR\rLS': 'ST Girls',
}, inplace=True)

In [ ]:
df_ap.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'INST\r_RE\rG', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS',
       'BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS', 'BC-E Boys', 'BC-E Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts["Institution Registered"] = "AU"

In [ ]:
df_ap.rename(columns={"INST\r_RE\rG" : "Institution Registered"}, inplace=True)

In [ ]:
df_ap_new = df_ap.drop(columns=('A_RE\rG'))

In [ ]:
df_ts_new = df_ts.drop(columns=['BC_B \nBOYS', 'BC_B \nGIRLS', 'BC_C \nBOYS', 'BC_C \nGIRLS','BC_D \nBOYS', 'BC_D \nGIRLS'])

In [ ]:
df_ap_new = df_ap.drop(columns=['BCB_B\rOYS', 'BCB_GI\rRLS', 'BCC_B\rOYS','BCC_GI\rRLS', 'BCD_B\rOYS', 'BCD_GI\rRLS'])

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'A_RE\rG', 'Branch Code',
       'OC Boys', 'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'BC-A Boys', 'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys',
       'EWS Girls', 'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=['A_RE\rG'], inplace=True)

In [ ]:
df_ap_new.columns

Index(['SNO', 'Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
df_ap_new.drop(columns=["SNO"], inplace=True)

In [ ]:
df_ap_new.columns

Index(['Institute Code', 'Institute Name', 'College Type',
       'Institution Registered', 'District Code', 'Location', 'Co-Ed Status',
       'Affiliated To', 'Established Year', 'Branch Code', 'OC Boys',
       'OC Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls', 'BC-A Boys',
       'BC-A Girls', 'BC-E Boys', 'BC-E Girls', 'EWS Boys', 'EWS Girls',
       'Tuition Fee', 'State'],
      dtype='object')

In [ ]:
df_ts_new.columns

Index(['Institute Code', 'Institute Name', 'Location', 'District Code',
       'Co-Ed Status', 'College Type', 'Established Year', 'Branch Code',
       'Branch Name', 'OC Boys', 'OC Girls', 'BC-A Boys', 'BC-A Girls',
       'BC-E Boys', 'BC-E Girls', 'SC Boys', 'SC Girls', 'ST Boys', 'ST Girls',
       'EWS Boys', 'EWS Girls', 'Tuition Fee', 'Affiliated To', 'State',
       'Institution Registered'],
      dtype='object')

In [ ]:
import pandas as pd
common_cols = list(set(df_ap_new.columns) & set(df_ts_new.columns))
df_combined = pd.concat([df_ap_new[common_cols], df_ts_new[common_cols]], ignore_index=True)

In [ ]:
df_combined.columns

Index(['Tuition Fee', 'OC Girls', 'SC Boys', 'Established Year',
       'Institute Code', 'State', 'Affiliated To', 'BC-E Girls',
       'College Type', 'Co-Ed Status', 'BC-A Boys', 'BC-A Girls', 'OC Boys',
       'SC Girls', 'Location', 'Branch Code', 'Institution Registered',
       'ST Boys', 'Institute Name', 'BC-E Boys', 'EWS Boys', 'District Code',
       'ST Girls', 'EWS Girls'],
      dtype='object')

In [ ]:
len(df_ap_new)

1572

In [ ]:
len(df_ts_new)

958

In [ ]:
len(df_combined)

2530

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
Tuition Fee,59
OC Girls,1
SC Boys,177
Established Year,0
Institute Code,0
State,0
Affiliated To,0
BC-E Girls,0
College Type,0
Co-Ed Status,0


In [ ]:
df_combined = df_combined.fillna(0)

In [ ]:
nan_counts = df_combined.isna().sum()
nan_counts

,0
Tuition Fee,0
OC Girls,0
SC Boys,0
Established Year,0
Institute Code,0
State,0
Affiliated To,0
BC-E Girls,0
College Type,0
Co-Ed Status,0


In [ ]:
df_combined.to_csv('college_list.csv')

In [ ]:
# 'Inst\n Code' = 'INSTCODE'
# 'Institute Name' = 'NAME OF THE INSTITUTION'
# 'College Type' = 'TYPE'
# 'Place' = 'PLACE'
# 'Dist \nCode' = 'DIST'
# 'Co Education' = 'COED'
# 'Year of Estab' = 'ESTD'
# 'Branch Code' = 'branch_\rcode'
# 'Branch Name'
# 'Tuition Fee' = 'Unnamed: 30'
# 'Affiliated To' = 'AFFL.'
# 'EWS \nGEN OU', 'EWS \nGIRLS OU', = 'OC_EWS_B\rOYS', 'OC_EWS_G\rIRLS'
# 'BCA_B\rOYS', 'BCA_GI\rRLS' = 'BC_A \nBOYS', 'BC_A \nGIRLS'
# 'BCE_B\rOYS', 'BCE_GI\rRLS' = 'BC_E \nBOYS', 'BC_E \nGIRLS'

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader

# Load Data
loader = CSVLoader("college_list.csv")
docs = loader.load()

# Convert text to embeddings using Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store in ChromaDB
vector_db = Chroma.from_documents(docs, embedding_model, persist_directory="vector_db")
vector_db.persist()
print("✅ Embeddings stored successfully in ChromaDB!")

<ipython-input-33-91b4b375ff45>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embeddings stored successfully in ChromaDB!


<ipython-input-33-91b4b375ff45>:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load embeddings from ChromaDB
vector_db = Chroma(persist_directory="vector_db", embedding_function=embedding_model)

# Convert into FAISS retriever
faiss_index = FAISS.from_documents(docs, embedding_model)

# Store FAISS index
faiss_index.save_local("vector_db/college_faiss_index")

print("✅ FAISS Index Created!")

✅ FAISS Index Created!


In [ ]:
import os
from groq import Groq

# 🔹 Set your API key
os.environ["GROQ_API_KEY"] = "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"

# 🔹 Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [ ]:
def retrieve_from_chromadb(query, collection, k=5):
    query_embedding = embedding_model.embed_query([query])  # Convert query to embedding
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k
    )
    return results["metadatas"]  # Returns top-k similar results

In [ ]:
def generate_answer(query, vector_db):
    # 🔹 Get query embedding
    query_embedding = embedding_model.embed_query(query)

    # 🔹 Retrieve the most relevant documents
    retrieved_docs = vector_db.similarity_search_by_vector(query_embedding, k=5)

    # 🔹 Extract relevant text
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # 🔹 Prepare the prompt for LLM
    prompt = f"Query: {query}\nAnswer: Not Available" if '0' in str(context) or not context.strip() else f"Use the following information to answer the query:\n{context}\n\nQuery: {query}\nAnswer:"

    # 🔹 Send request to Groq API
    response = client.chat.completions.create(
        model="qwen-2.5-32b",  # Example: Choose your model
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )

    return response

In [44]:
query = "List Top Colleges in State Andhra Pradesh"
response = generate_answer(query, vector_db)
print(response.choices[0].message.content)

Here is a list of some top colleges in the state of Andhra Pradesh:

1. **Indian Institute of Technology Kharagpur** - Oddur - One of the IITs in India, it has a campus in Andhra Pradesh.
2. **Andhra University, Visakhapatnam** - One of the oldest and prestigious universities in the region.
3. **VIT University (Vellore Institute of Technology) – Vizag Campus** - A well-known private university with a campus in Visakhapatnam.
4. **JNTUA (Jawaharlal Nehru Technological University Anantapur)** - Known for engineering and technology courses.
5. **AU College of Engineering, Visakhapatnam** - A part of Andhra University and one of the top engineering colleges in the state.
6. **RVR & JC College of Engineering** - Located in Guntur and is highly regarded for its engineering programs.
7. **Gayatri Vidya Parishad College of Engineering** - Situated in Visakhapatnam, renowned for engineering and technology programs.
8. **Marutiruchi Subba Raju Institute of Technology (MSRIT)** - Located in Kakin

---

---